In [1]:
import syft as sy
from syft import ActionObject
from syft import syft_function, syft_function_single_use
from time import sleep

kj/filesystem-disk-unix.c++:1703: warning: PWD environment variable doesn't match current directory; pwd = /Users/koen/workspace/pysyft


In [2]:
# node = sy.orchestra.launch(name="test-domain-helm", port=8800, dev_mode=True, reset=True, tail=True)

with server

In [3]:
# client = sy.login(port=8800, email="info@openmined.org", password="changethis")

Without server

In [4]:
node = sy.orchestra.launch(name="test-domain-helm2", dev_mode=True, reset=True, n_consumers=0)
client = node.login(email="info@openmined.org", password="changethis")

CREATING A PRODUCER ON tcp://127.0.0.1:56761
Logged into <test-domain-helm2: High side Domain> as <info@openmined.org>


SyftWarning: You are using a default password. Please change the password using `[your_client].me.set_password([new_password])`.

In [5]:
node.python_node.queue_config

In [6]:
from syft.service.queue.zmq_queue import ZMQQueueConfig, ZMQClientConfig

In [7]:
worker_node = sy.orchestra.launch(name="worker-node-helm1", dev_mode=True, reset=True,
                                  n_consumers=3,
                                  queue_config=ZMQQueueConfig(client_config=ZMQClientConfig(create_producer=False)))


CREATING A CONSUMER ON tcp://127.0.0.1:None
spawning thread
CREATING A CONSUMER ON tcp://127.0.0.1:None
spawning thread
CREATING A CONSUMER ON tcp://127.0.0.1:None
spawning thread


In [8]:
# from gevent import monkey

In [9]:
# monkey.patch_all(thread=True)

In [10]:
# type(node.python_node.queue_manager.consumers["api_call"])

In [11]:
# node.python_node.queue_manager.consumers["api_call"][0].address

setup: compute train-test overlap between a very large train set and a smaller test set. Small test is still to big for memory, so we split it into 54 parts. We keep 1 of those parts in memory. We dont keep the train set in memory, but read and compare with 1/54 parts line by line. Each part takes ~30 hours, but we can run 54 processes in parallel.

# Setup syft functions

## Dataset

In [12]:
x = ActionObject.from_obj([1,2])
x_ptr = x.send(client)

## Batch function

In [13]:
@syft_function()
def process_batch(batch):
    # takes 30 hours normally
    print(f"starting batch {batch}")
    from time import sleep
    sleep(1)
    print("done")
    return batch+1

SyftSuccess: Syft function 'process_batch' successfully created. To add a code request, please create a project using `project = syft.Project(...)`, then use command `project.create_code_request`.

In [14]:
client.code.submit(process_batch)

SyftSuccess: User Code Submitted

## Main function

In [15]:
@syft_function_single_use(x=x_ptr)
def process_all(domain, x):
    jobs = []
    print("Launching jobs")
    for elem in x:
        # We inject a domain object in the scope
        batch_job = domain.launch_job(process_batch, batch=elem)
        jobs += [batch_job]
    print("starting aggregation")
    print("Done")
    results = [x.wait().get() for x in jobs]
#     return 1
    return sum(results)

SyftSuccess: Syft function 'process_all' successfully created. To add a code request, please create a project using `project = syft.Project(...)`, then use command `project.create_code_request`.

# Approve & run

In [16]:
client.code.request_code_execution(process_all)
client.requests[-1].approve()

Request approved for domain test-domain-helm2


SyftSuccess: Request b1d451b802dd4983a60cf701faef5339 changes applied

In [17]:
job = client.code.process_all(x=x_ptr, blocking=False)
sleep(1)

FUNCTION LOG: Launching jobs
FUNCTION LOG: starting aggregation
FUNCTION LOG: Done
FUNCTION LOG: starting batch 1
FUNCTION LOG: starting batch 2
FUNCTION LOG: done
result 2
FUNCTION LOG: done
result 3


In [18]:
job.wait().get()

result 5


5

In [19]:
job.subjobs[0].logs()

starting batch 1



##### Next steps



- [x] seaweedfs
- - [wip] merge into normal code
- [x] Multiple consumers (queues)
- - [x] implement messagequeue
- - [x] integrate messagequeue with the rest of the code
- [x] test flow with async main function
- [x]  up datastructures for jobs and logs
- - [x] set up reprs
- [wip] integrate helm code
- [ ] test on large machine